In [2]:
import pandas as pd
import numpy as np
import os
import pickle
import json
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns

# ============================================
# CONFIGURATION OPTIMISÉE POUR MOBILE
# ============================================

DATASET_PATH = r"C:\Users\sersi\Desktop\projet_SE_et_IOT\HandSense_project\asl_dataset_complete.csv"
MODEL_OUTPUT_DIR = r"C:\Users\sersi\Desktop\projet_SE_et_IOT\HandSense_project\model_mobile"

FEATURE_COLUMNS = [
    'flex_1', 'flex_2', 'flex_3', 'flex_4', 'flex_5',
    'GYRx', 'GYRy', 'GYRz',
    'ACCx', 'ACCy', 'ACCz'
]

TEST_SIZE = 0.2 
RANDOM_STATE = 42

# ⚡ PARAMÈTRES OPTIMISÉS POUR MOBILE
N_ESTIMATORS = 30        # ✅ Réduit de 100 → 30 (3x plus léger)
MAX_DEPTH = 12           # ✅ Limite la profondeur des arbres
MIN_SAMPLES_SPLIT = 10   # ✅ Évite le surapprentissage
MIN_SAMPLES_LEAF = 4     # ✅ Réduit la complexité

# ============================================
# FONCTION 1: CHARGER LES DONNÉES
# ============================================

def load_and_explore_data():
    print("="*60)
    print("📊 CHARGEMENT DES DONNÉES")
    print("="*60)
    
    df = pd.read_csv(DATASET_PATH)
    
    print(f"\n✅ Dataset chargé avec succès!")
    print(f"   📏 Dimensions: {df.shape[0]} lignes × {df.shape[1]} colonnes")
    print(f"\n🏷️  Classes: {df['label'].nunique()}")
    print(f"   {sorted(df['label'].unique())}")
    
    print(f"\n📈 RÉPARTITION PAR CLASSE:")
    label_counts = df['label'].value_counts().sort_index()
    for label, count in label_counts.items():
        print(f"   {label:15s}: {count:6d} échantillons")
    
    missing = df[FEATURE_COLUMNS].isnull().sum()
    if missing.sum() > 0:
        print(f"\n⚠️  VALEURS MANQUANTES:")
        print(missing[missing > 0])
    else:
        print(f"\n✅ Aucune valeur manquante!")
    
    return df


# ============================================
# FONCTION 2: PRÉTRAITEMENT
# ============================================

def preprocess_data(df):
    print("\n" + "="*60)
    print("🔧 PRÉTRAITEMENT DES DONNÉES")
    print("="*60)
    
    X = df[FEATURE_COLUMNS].values
    y = df['label'].values
    
    print(f"\n📊 Features (X): {X.shape}")
    print(f"🏷️  Labels (y): {y.shape}")
    
    # Encoder les labels
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)
    
    print(f"\n🔢 Encodage des labels:")
    for i, label in enumerate(label_encoder.classes_[:5]):
        print(f"   '{label}' → {i}")
    print(f"   ... ({len(label_encoder.classes_)} classes au total)")
    
    # Normaliser
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    print(f"\n✅ Normalisation appliquée (StandardScaler)")
    
    # Split train/test
    X_train, X_test, y_train, y_test = train_test_split(
        X_scaled, y_encoded, 
        test_size=TEST_SIZE, 
        random_state=RANDOM_STATE,
        stratify=y_encoded
    )
    
    print(f"\n📊 SÉPARATION TRAIN/TEST:")
    print(f"   🎯 Train: {X_train.shape[0]} échantillons")
    print(f"   🧪 Test:  {X_test.shape[0]} échantillons")
    
    return X_train, X_test, y_train, y_test, scaler, label_encoder


# ============================================
# FONCTION 3: ENTRAÎNER LE MODÈLE OPTIMISÉ
# ============================================

def train_model(X_train, y_train):
    print("\n" + "="*60)
    print("🚀 ENTRAÎNEMENT DU MODÈLE OPTIMISÉ MOBILE")
    print("="*60)
    
    # ⚡ Modèle optimisé pour mobile
    model = RandomForestClassifier(
        n_estimators=N_ESTIMATORS,           # Moins d'arbres
        max_depth=MAX_DEPTH,                 # Profondeur limitée
        min_samples_split=MIN_SAMPLES_SPLIT, # Nœuds plus larges
        min_samples_leaf=MIN_SAMPLES_LEAF,   # Feuilles plus larges
        random_state=RANDOM_STATE,
        n_jobs=-1,
        verbose=1
    )
    
    print(f"\n🌳 Configuration du modèle:")
    print(f"   📦 Nombre d'arbres: {N_ESTIMATORS} (au lieu de 100)")
    print(f"   📏 Profondeur max: {MAX_DEPTH}")
    print(f"   🔹 Min samples split: {MIN_SAMPLES_SPLIT}")
    print(f"   🔹 Min samples leaf: {MIN_SAMPLES_LEAF}")
    print(f"\n   ✅ Optimisé pour réduire la taille du modèle ONNX")
    
    print(f"\n⏳ Entraînement en cours...")
    model.fit(X_train, y_train)
    
    print(f"✅ Entraînement terminé!")
    
    # Validation croisée
    print(f"\n🔄 Validation croisée (5-fold)...")
    cv_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy')
    
    print(f"   Scores: {[f'{s:.4f}' for s in cv_scores]}")
    print(f"   Moyenne: {cv_scores.mean():.4f} (±{cv_scores.std():.4f})")
    
    return model


# ============================================
# FONCTION 4: ÉVALUER LE MODÈLE
# ============================================

def evaluate_model(model, X_test, y_test, label_encoder):
    print("\n" + "="*60)
    print("📈 ÉVALUATION DU MODÈLE")
    print("="*60)
    
    y_pred = model.predict(X_test)
    
    accuracy = accuracy_score(y_test, y_pred)
    print(f"\n🎯 ACCURACY: {accuracy:.4f} ({accuracy*100:.2f}%)")
    
    print(f"\n📊 RAPPORT DE CLASSIFICATION:")
    print("-"*60)
    report = classification_report(
        y_test, y_pred, 
        target_names=label_encoder.classes_,
        zero_division=0
    )
    print(report)
    
    cm = confusion_matrix(y_test, y_pred)
    
    return accuracy, report, cm


# ============================================
# FONCTION 5: VISUALISATION
# ============================================

def visualize_results(cm, label_encoder, feature_importance, model_dir):
    print("\n" + "="*60)
    print("📊 CRÉATION DES VISUALISATIONS")
    print("="*60)
    
    # Matrice de confusion
    plt.figure(figsize=(12, 10))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=label_encoder.classes_,
                yticklabels=label_encoder.classes_)
    plt.title('Matrice de Confusion (Modèle Mobile)')
    plt.ylabel('Vraie classe')
    plt.xlabel('Classe prédite')
    plt.xticks(rotation=45, ha='right')
    plt.yticks(rotation=0)
    plt.tight_layout()
    
    cm_path = os.path.join(model_dir, 'confusion_matrix_mobile.png')
    plt.savefig(cm_path, dpi=150)
    print(f"   ✅ Matrice de confusion: {cm_path}")
    plt.close()
    
    # Importance des features
    plt.figure(figsize=(10, 6))
    feature_names = FEATURE_COLUMNS
    indices = np.argsort(feature_importance)[::-1]
    
    plt.bar(range(len(feature_importance)), feature_importance[indices])
    plt.xticks(range(len(feature_importance)), 
               [feature_names[i] for i in indices], 
               rotation=45, ha='right')
    plt.title('Importance des Features (Modèle Mobile)')
    plt.ylabel('Importance')
    plt.xlabel('Features')
    plt.tight_layout()
    
    fi_path = os.path.join(model_dir, 'feature_importance_mobile.png')
    plt.savefig(fi_path, dpi=150)
    print(f"   ✅ Importance des features: {fi_path}")
    plt.close()


# ============================================
# FONCTION 6: SAUVEGARDER LE MODÈLE
# ============================================

def save_model(model, scaler, label_encoder, model_dir):
    print("\n" + "="*60)
    print("💾 SAUVEGARDE DU MODÈLE MOBILE")
    print("="*60)
    
    os.makedirs(model_dir, exist_ok=True)
    
    # Sauvegarder le modèle
    model_path = os.path.join(model_dir, 'asl_model_mobile.pkl')
    with open(model_path, 'wb') as f:
        pickle.dump(model, f)
    
    # Taille du fichier
    size_mb = os.path.getsize(model_path) / (1024 * 1024)
    print(f"   ✅ Modèle: {model_path}")
    print(f"      📦 Taille: {size_mb:.2f} MB")
    
    # Scaler
    scaler_path = os.path.join(model_dir, 'scaler.pkl')
    with open(scaler_path, 'wb') as f:
        pickle.dump(scaler, f)
    print(f"   ✅ Scaler: {scaler_path}")
    
    # Label encoder
    encoder_path = os.path.join(model_dir, 'label_encoder.pkl')
    with open(encoder_path, 'wb') as f:
        pickle.dump(label_encoder, f)
    print(f"   ✅ Label Encoder: {encoder_path}")
    
    # Métadonnées
    metadata = {
        'feature_columns': FEATURE_COLUMNS,
        'classes': label_encoder.classes_.tolist(),
        'n_classes': len(label_encoder.classes_),
        'n_features': len(FEATURE_COLUMNS),
        'model_type': 'RandomForestClassifier',
        'n_estimators': N_ESTIMATORS,
        'max_depth': MAX_DEPTH,
        'optimized_for': 'mobile',
        'target_format': 'ONNX'
    }
    
    metadata_path = os.path.join(model_dir, 'model_metadata.json')
    with open(metadata_path, 'w') as f:
        json.dump(metadata, f, indent=4)
    print(f"   ✅ Métadonnées: {metadata_path}")


# ============================================
# FONCTION 7: TEST DE PRÉDICTION
# ============================================

def test_prediction(model, scaler, label_encoder, X_test, y_test):
    print("\n" + "="*60)
    print("🧪 TEST DE PRÉDICTION")
    print("="*60)
    
    idx = np.random.randint(0, len(X_test))
    sample = X_test[idx:idx+1]
    true_label = label_encoder.classes_[y_test[idx]]
    
    prediction = model.predict(sample)
    predicted_label = label_encoder.classes_[prediction[0]]
    
    probabilities = model.predict_proba(sample)[0]
    
    print(f"\n📊 Échantillon #{idx}:")
    print(f"   Features: {sample[0][:5]}...")
    print(f"\n🎯 Vraie classe: {true_label}")
    print(f"🤖 Prédiction: {predicted_label}")
    print(f"   {'✅ CORRECT' if true_label == predicted_label else '❌ INCORRECT'}")
    
    print(f"\n📊 Top 3 probabilités:")
    top_3_idx = np.argsort(probabilities)[-3:][::-1]
    for i, idx in enumerate(top_3_idx, 1):
        label = label_encoder.classes_[idx]
        prob = probabilities[idx]
        print(f"   {i}. {label:15s}: {prob:.4f} ({prob*100:.2f}%)")


# ============================================
# FONCTION PRINCIPALE
# ============================================

def main():
    print("\n" + "="*70)
    print("🤖 ENTRAÎNEMENT DU MODÈLE ASL OPTIMISÉ POUR MOBILE")
    print("="*70)
    
    # 1. Charger
    df = load_and_explore_data()
    
    # 2. Prétraiter
    X_train, X_test, y_train, y_test, scaler, label_encoder = preprocess_data(df)
    
    # 3. Entraîner (modèle optimisé)
    model = train_model(X_train, y_train)
    
    # 4. Évaluer
    accuracy, report, cm = evaluate_model(model, X_test, y_test, label_encoder)
    
    # 5. Créer le dossier
    os.makedirs(MODEL_OUTPUT_DIR, exist_ok=True)
    
    # 6. Visualiser
    feature_importance = model.feature_importances_
    visualize_results(cm, label_encoder, feature_importance, MODEL_OUTPUT_DIR)
    
    # 7. Sauvegarder
    save_model(model, scaler, label_encoder, MODEL_OUTPUT_DIR)
    
    # 8. Test
    test_prediction(model, scaler, label_encoder, X_test, y_test)
    
    print("\n" + "="*70)
    print("✨ ENTRAÎNEMENT TERMINÉ AVEC SUCCÈS !")
    print("="*70)
    print(f"\n📊 Résumé:")
    print(f"   🎯 Accuracy: {accuracy:.4f} ({accuracy*100:.2f}%)")
    print(f"   📁 Modèle sauvegardé: {MODEL_OUTPUT_DIR}")
    print(f"   📦 Optimisé pour conversion ONNX")
    print(f"   🌳 Arbres: {N_ESTIMATORS} (réduit pour mobile)")
    print(f"   📏 Profondeur max: {MAX_DEPTH}")


if __name__ == "__main__":
    main()


🤖 ENTRAÎNEMENT DU MODÈLE ASL OPTIMISÉ POUR MOBILE
📊 CHARGEMENT DES DONNÉES

✅ Dataset chargé avec succès!
   📏 Dimensions: 1500000 lignes × 13 colonnes

🏷️  Classes: 40
   ['a', 'b', 'bad', 'c', 'd', 'deaf', 'e', 'f', 'fine', 'g', 'good', 'goodbye', 'h', 'hello', 'hungry', 'i', 'j', 'k', 'l', 'm', 'me', 'n', 'no', 'o', 'p', 'please', 'q', 'r', 's', 'sorry', 't', 'thankyou', 'u', 'v', 'w', 'x', 'y', 'yes', 'you', 'z']

📈 RÉPARTITION PAR CLASSE:
   a              :  37500 échantillons
   b              :  37500 échantillons
   bad            :  37500 échantillons
   c              :  37500 échantillons
   d              :  37500 échantillons
   deaf           :  37500 échantillons
   e              :  37500 échantillons
   f              :  37500 échantillons
   fine           :  37500 échantillons
   g              :  37500 échantillons
   good           :  37500 échantillons
   goodbye        :  37500 échantillons
   h              :  37500 échantillons
   hello          :  37500 écha

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:   46.3s finished


✅ Entraînement terminé!

🔄 Validation croisée (5-fold)...


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:   34.4s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  30 out of  30 | elapsed:    1.2s finished
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:   45.8s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  30 out of  30 | elapsed:    1.4s finished
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:   37.1s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  30 out of  30 | elapsed:    1.4s finished
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_j

   Scores: ['0.9115', '0.9162', '0.9142', '0.9126', '0.9095']
   Moyenne: 0.9128 (±0.0023)

📈 ÉVALUATION DU MODÈLE


[Parallel(n_jobs=4)]: Done  30 out of  30 | elapsed:    2.0s finished



🎯 ACCURACY: 0.9098 (90.98%)

📊 RAPPORT DE CLASSIFICATION:
------------------------------------------------------------
              precision    recall  f1-score   support

           a       0.89      0.98      0.93      7500
           b       0.94      0.99      0.96      7500
         bad       0.83      0.68      0.75      7500
           c       0.97      0.81      0.88      7500
           d       0.76      0.94      0.84      7500
        deaf       0.87      0.84      0.86      7500
           e       0.81      0.91      0.86      7500
           f       1.00      1.00      1.00      7500
        fine       0.88      0.95      0.91      7500
           g       0.94      1.00      0.97      7500
        good       0.92      0.65      0.76      7500
     goodbye       0.88      0.86      0.87      7500
           h       1.00      1.00      1.00      7500
       hello       0.84      0.84      0.84      7500
      hungry       0.99      0.96      0.98      7500
           i   

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  30 out of  30 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  30 out of  30 | elapsed:    0.0s finished
